In [ ]:
#mount the google drive first
#https://www.youtube.com/watch?v=LNwODJXcvt4 source to train yolov8 model 
from google.colab import drive
drive.mount('content/drive')

In [ ]:
!pip install -q ultralytics roboflow

In [ ]:
#set the roboflow api key
import os 
ROBOFLOW_API_KEY = "ENTER_YOUR_OWN_API_KEY"
os.environ["ROBOFLOW_API_KEY"] = ROBOFLOW_API_KEY

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key=os.environ["ROBOFLOW_API_KEY"])

#dataset user and dataset name workspace
#https://universe.roboflow.com/cvsense/basketball_detection-lbacg/dataset/3 dataset I used with 4546 images alot 

#bigger dataset
proj = rf.workspace("cvsense").project("basketball_detection-lbacg")
dataset = proj.version(3).download("yolov8")

#https://universe.roboflow.com/034-ganesh-kumar-m-v-cs-r2lwe/basketball-lhqoe/browse?queryText=&pageSize=50&startingIndex=0&browseQuery=true smaller datset for first try
#smaller datset
# proj = rf.workspace("034-ganesh-kumar-m-v-cs-r2lwe").project("basketball-lhqoe")
# dataset = proj.version(1).download("yolov8")




In [ ]:
#then after download dataset, update the train test and validation set in the yaml config file

In [ ]:
import yaml, pathlib
DATA_YAML = "/content/Basketball_Detection-3/data.yaml"
names = yaml.safe_load(open(DATA_YAML))["names"]   # e.g. ['ball','made','person','rim','shoot']
print("names:", names)

# robust mapping (handles synonyms)
def find_id(options):
    for i,n in enumerate(names):
        if n.lower() in options: return i
    raise ValueError(f"None of {options} in names: {names}")

BALL_ID = find_id({"ball","basketball"})
RIM_ID  = find_id({"rim","basketball hoop","hoop"})
print("BALL_ID =", BALL_ID, "RIM_ID =", RIM_ID)
#for the big dataset, ball class = 0, rim class = 3

In [ ]:
#train the yolov9 model with our own dataset with the dataset loaded in 
!yolo task=detect mode=train model=yolov8m.pt data={dataset.location}/data.yaml classes=[0, 3] epochs=20 imgsz=640

In [ ]:
#confusion matrix from training
from IPython.display import Image, display
Image(filename=f'/content/runs/detect/train/confusion_matrix.png', width=600)

In [ ]:
#training results graphs 
Image(filename=f'/content/runs/detect/train/results.png', width=600)

In [ ]:
#choose the best model that we have during training and specify the data location of our validation set and run over the validation set 
!yolo task=detect mode=val model=/content/runs/detect/train/weights/best.pt data={dataset.location}/data.yaml

In [ ]:
#now perform prediction on the trained model on the test set
!yolo task=detect mode=predict model=/content/runs/detect/train/weights/best.pt conf=0.5 source={dataset.location}/test/images

In [ ]:
import glob
i =0
for image_path in glob.glob(f'/content/runs/detect/predict2/*.jpg'):
    if i == 10:
        break
    display(Image(filename=image_path, height=600))
    print("/n")
    i+=1